In [ ]:
from dotenv import load_dotenv
from litellm import completion

_ = load_dotenv()

In [ ]:
# "gemini/gemini-1.5-flash-latest"
MODEL_GPT_41 = "openai/gpt-4.1"
messages = [
    {"role": "user", "content": "Are you ready"}
]

response = completion(
    model=MODEL_GPT_41,
    messages=messages
)

In [ ]:
# print(response.choices[0].message.content)  # Yes, I'm ready! How can I help you today?

In [ ]:
prompt_text = f"""
Given the following earnings call transcript, propose a knowledge graph schema. The schema should be represented using nodes and relationships.

Your proposal should define:
1.  **Node Types:** The different types of entities (e.g., Company, Executive, Analyst, FinancialMetric, Quarter) you'd expect to find. For each node type, list the key properties that would describe it.
2.  **Relationship Types:** The different relationships that connect the nodes (e.g., REPORTS_FINANCIALS_FOR, DISCUSSES_METRIC, ASKS_QUESTION_TO). For each relationship, specify the start and end node types and any properties the relationship itself should have.

The goal is to design a schema that can be applied to a series of similar earnings transcripts to track financial performance,
strategic decisions, and the roles of key individuals over time.
Focus on the core entities and relationships needed to capture the main information.
Your answer should be headed "Knowledge Graph Schema Proposal" (verbatim) and should be just schema, i.e. no endnotes, no visualization examples.

---
[START OF FILE CONTENT]
{file_content}
[END OF FILE CONTENT]
---

Knowledge Graph Schema Proposal:
"""

In [ ]:
with open("../text-kb/2023-q3-earnings-transcript.md", "r", encoding="utf-8") as file:
    file_content = file.read()

# len(file_content) 47775

In [ ]:
messages = [
    {"role": "user", "content": prompt_text}
]

In [ ]:
%%time

response = completion(
        model=MODEL_GPT_41,
        messages=messages
)
# Wall time: 16.9-15.7 s

In [ ]:
#print(response.choices[0].message.content)

In [ ]:
KNOWLEDGE_GRAPH_SCHEMA_proposal = response.choices[0].message.content  # see KG-construction-1a.txt

In [ ]:
#file_content

In [ ]:
revision_prompt = f"""
Given the following existing knowledge graph schema and a new document, propose amendments to the schema to better accommodate information from the new document.

Your response must be a concise, bulleted list of suggested changes under a single heading "Feedback:". Do not generate any other text. Each bullet point should be specific, detailing what to add or modify in the schema.

---
{KNOWLEDGE_GRAPH_SCHEMA_proposal}
---

[START OF NEW FILE CONTENT]
{file_content}
[END OF NEW FILE CONTENT]
---

Feedback:
- Suggested additions to Node Types:
- Suggested additions to Relationship Types:
- Suggested new properties for existing Node Types or Relationship Types:
"""

In [ ]:
extra_files = [
    "6 things you can do with Google Photos’ Create tab"
    ,"AI Mode in Google Search expands to more languages"
    ,"Alphabet to Present at the Goldman Sachs 2025 Communacopia + Technology Conference - Alphabet Investor Relations"
    ,"Google's statement on Sept 2025 Search DOJ decision"
]
with open(f"../text-kb/{extra_files[3]}.md", "r", encoding="utf-8") as file:
    file_content = file.read()

# len(file_content)  # 2011

In [ ]:
prompt_text = revision_prompt
messages = [
    {"role": "user", "content": prompt_text}
]

In [ ]:
%%time

response = completion(
        model=MODEL_GPT_41,
        messages=messages
)
# Wall time: 3 s

In [ ]:
#print(response.choices[0].message.content)
#revision_prompt

In [ ]:
feedback3 = response.choices[0].message.content

In [ ]:
with open(f"../text-kb/{extra_files[0]}.md", "r", encoding="utf-8") as file:
    file_content = file.read()

# len(file_content)  # 1623

In [ ]:
%%time
prompt_text = revision_prompt
messages = [
    {"role": "user", "content": prompt_text}
]
response = completion(
        model=MODEL_GPT_41,
        messages=messages
)
# Wall time: 8.28 s
feedback0 = response.choices[0].message.content

In [ ]:
%%time
with open(f"../text-kb/{extra_files[1]}.md", "r", encoding="utf-8") as file:
    file_content = file.read()

len(file_content)  # 
prompt_text = revision_prompt  # re-evalute using new {file_content} (and same KNOWLEDGE_GRAPH_SCHEMA_proposal)
messages = [
    {"role": "user", "content": prompt_text}
]
response = completion(
        model=MODEL_GPT_41,
        messages=messages
)
# Wall time: 6.28 s
feedback1 = response.choices[0].message.content

In [ ]:
#print(response.choices[0].message.content)  # see -feedback1.txt
#prompt_text
#feedback0